In [1]:
import pandas as pd
from tqdm.auto import tqdm
import warnings
from dev import *
from dev.aggregate import clear_aggregated_cache
from concurrent.futures import ProcessPoolExecutor, as_completed

warnings.filterwarnings("ignore")
clear_aggregated_cache()


# Worker function
def worker(eval_setup, dataset_name, source_name, attack_name, attack_strength):
    performance = get_performance(
        dataset_name,
        source_name,
        attack_name,
        attack_strength,
        mode=eval_setup,
    )
    quality = get_quality(
        dataset_name,
        source_name,
        attack_name,
        attack_strength,
        mode=eval_setup,
    )
    record = {
        "eval_setup": eval_setup,
        "dataset_name": dataset_name,
        "source_name": source_name,
        "attack_name": attack_name,
        "attack_strength": attack_strength,
    }
    record.update({k: v for k, v in performance.items()})
    record.update({k: v for k, v in quality.items()})
    return record


# Configuration for ThreadPoolExecutor
max_workers = 16  # Set the maximum number of workers
job_limit = float("inf")  # Set the job limit, can be set to float('inf') for no limit

# Prepare data for job submission
json_dict = get_all_json_paths(
    lambda _dataset_name, _attack_name, _attack_strength, _source_name, _result_type: (
        (_attack_name is not None)
        and (_attack_strength is not None)
        and (not _source_name.startswith("real"))
    )
)
record_keys = sorted(
    list(set([(key[0], key[3], key[1], key[2]) for key in json_dict.keys()])),
    key=lambda x: tuple((item is None, item) for item in x),
)
total = min(len(EVALUATION_SETUPS) * len(record_keys), job_limit)

# Parallel execution
records = []
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for eval_setup in EVALUATION_SETUPS.keys():
        for dataset_name, source_name, attack_name, attack_strength in record_keys:
            if len(futures) < job_limit:
                future = executor.submit(
                    worker,
                    eval_setup,
                    dataset_name,
                    source_name,
                    attack_name,
                    attack_strength,
                )
                futures.append(future)
            else:
                break

    # Collect results
    for future in tqdm(as_completed(futures), total=total):
        records.append(future.result())

# Convert to DataFrame and save
table = pd.DataFrame(records)
table.to_pickle("all_result.pkl")

  0%|          | 0/3699 [00:00<?, ?it/s]

In [2]:
df = pd.read_pickle("all_result.pkl")
df = df[df["eval_setup"] == "removal"]
df[df.isna().any(axis=1)]

,eval_setup,dataset_name,source_name,attack_name,attack_strength,acc_1,auc_1,low100_1,low1000_1,acc_100,...,clip_fid,psnr,ssim,nmi,lpips,watson,aesthetics,artifacts,clip_score,normalized_quality


In [3]:
df = pd.read_pickle("all_result.pkl")
df = df[df["eval_setup"] == "removal"]

for dataset_name in DATASET_NAMES.keys():
    for source_name in WATERMARK_METHODS.keys():
        print(
            len(
                df[
                    (df["dataset_name"] == dataset_name)
                    & (df["source_name"] == source_name)
                ]
            )
        )

print(len(df), 137 * 9)

137
137
137
137
137
137
137
137
137
1233 1233
